In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
groq_api = os.getenv('groq_api')
print(groq_api)

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


In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api,
    model="llama3-70b-8192"
)

response = llm.invoke("The first person to land on moon was?")
print(response.content)

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the Moon.


In [4]:
import chromadb
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="default_collection")

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.lever.co/mindtickle/011afcfc-09da-48f6-8acc-d49a13791327/apply?source=LinkedIn")
page_data = loader.load().pop().page_content

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
page_data

"Mindtickle - SDE-I (Frontend)SDE-I (Frontend)BangaloreEngineering – Engineering /Full-time /On-siteSubmit your applicationLinkedIn profileLoading...Apply with LinkedInProfile addedYour full LinkedIn profile will beshared. Learn moreResume/CV ✱ATTACH RESUME/CVCouldn't auto-read resume.Analyzing resume...Success!File exceeds the maximum upload size of 100MB. Please try a smaller size.Full name✱Email✱Phone ✱Current location No location found. Try entering a different locationLoadingCurrent company LinksLinkedIn URLTwitter URLGitHub URLPortfolio URLOther websiteSDE (Frontend)-BangaloreYour current location✱Are you open to the Bangalore location?✱What is your CTC (Breakup)✱What is your Expected CTC?✱What is your Notice Period?✱Do you currently have any job offers? If yes, please provide details.✱Is there anything else you'd like to share with us?✱How many years of relevant experience do you have in Frontend Development?✱Additional informationYes, Mindtickle can contact me about future job 

In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [8]:
prompt_extract = PromptTemplate.from_template( 
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and get relevant content which include 
    the following: `role`, `experience`, `skills`, `description`, and `company information`.
    Please make sure that the response should be in valid JSON format and should not contain any preamble.
    ### VALID JSON RESPONSE:
    """
)
chain_extract = prompt_extract | llm
response = chain_extract.invoke(input={'page_data': page_data})
json_parser = JsonOutputParser()
job_info = json_parser.parse(response.content)

In [9]:
import fitz 
def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

resume_text = read_pdf("./Anurag_Pathak_SDE_Resume.pdf")


In [10]:
import chromadb
import uuid
chroma_client = chromadb.PersistentClient('vectorstore')
collection = chroma_client.get_or_create_collection(name="resumes")
if not collection.count():
    collection.add(documents=resume_text, ids=[str(uuid.uuid4)])


In [11]:
prompt_template = PromptTemplate.from_template(
    """
    ### RESUME:
    {resume_text}
    ### JOB POSTING:
    {job_posting}
    ### INSTRUCTION:
    You are job applicant, Based on the resume and job posting which is a JSON object with following keys: `role`, `experience`, `skills` and `description`.
    , generate a cold email (add subject and follow proper format of a E-Mail) for the job application highlighting your experiences (calculate form the resume), 
    skills and how can you contribute to the team. How your values and skills align with the company goals. 
    Make sure to describe any relevant projects which should align with the skills in the job posting
    Also make sure to add links (at the bottom after regards) of the portfolio, github, mailID, phone number or other relevant links required in proper format.
    please make sure not to exceed 450 words limit. do not provide a preamble.
    ### RESPONSE (NO PREAMBLE):    
    """
)
chain = prompt_template | llm
response = chain.invoke(input={"resume_text": resume_text, "job_posting": job_info})
print(response.content)

Subject: Application for SDE-I (Frontend) Role at Mindtickle

Dear Hiring Manager,

I am excited to apply for the SDE-I (Frontend) role at Mindtickle, as advertised on [Job Portal/Source]. With a strong background in Frontend Development and a passion for building scalable and efficient applications, I am confident that I can make a valuable contribution to your team.

As a recent graduate in Computer Science and Engineering, I have gained significant experience in Frontend Development through my internships and projects. My experience in developing and optimizing UI components using JavaScript, TypeScript, React, and Angular has equipped me with the skills to handle complex frontend tasks. In my previous role at GrapeCity, I improved the user experience of Wijmo Flexcharts by adding keyboard accessibility and optimized Wijmo Flexgrid components, resulting in a 20% performance enhancement.

I am particularly drawn to Mindtickle's focus on innovation and customer satisfaction. My values